## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-08-20-53-05 +0000,nypost,Zohran Mamdani’s mayoral campaign pays $28K to...,https://nypost.com/2025/10/08/us-news/zohran-m...
1,2025-10-08-20-52-01 +0000,wapo,The Pentagon’s revised press rules are unaccep...,https://www.washingtonpost.com/business/2025/1...
2,2025-10-08-20-51-45 +0000,nyt,Here is the latest.,https://www.nytimes.com/live/2025/10/08/us/tru...
3,2025-10-08-20-51-43 +0000,nypost,‘Cool pavement’ pilot program could come to NY...,https://nypost.com/2025/10/08/us-news/cool-pav...
4,2025-10-08-20-50-43 +0000,missionlocal,"At 16th and Albion, a defunct laundromat turns...",https://missionlocal.org/2025/10/at-16th-and-a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2375/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
118,trump,43
41,new,16
67,shutdown,15
229,gaza,15
107,fire,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
34,2025-10-08-19-36-45 +0000,latimes,Trump sends California National Guard to Illin...,https://www.latimes.com/politics/story/2025-10...,97
65,2025-10-08-18-33-00 +0000,wsj,President Trump called for the jailing of Chic...,https://www.wsj.com/politics/policy/trump-call...,97
84,2025-10-08-17-33-40 +0000,nyt,Trump Says Chicago Mayor and Illinois Governor...,https://www.nytimes.com/2025/10/08/us/trump-pr...,94
307,2025-10-07-23-53-21 +0000,nypost,Mother of US Navy medic rescued from Gaza in d...,https://nypost.com/2025/10/07/world-news/mothe...,93
277,2025-10-08-03-26-15 +0000,nyt,Trump Suggests Furloughed Federal Workers May ...,https://www.nytimes.com/2025/10/07/us/politics...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
34,97,2025-10-08-19-36-45 +0000,latimes,Trump sends California National Guard to Illin...,https://www.latimes.com/politics/story/2025-10...
168,71,2025-10-08-13-33-06 +0000,nypost,Ex-FBI Director James Comey insists he’s innoc...,https://nypost.com/2025/10/08/us-news/ex-fbi-d...
36,63,2025-10-08-19-34-04 +0000,nypost,House Speaker Mike Johnson rejects standalone ...,https://nypost.com/2025/10/08/us-news/house-sp...
27,58,2025-10-08-20-00-18 +0000,latimes,'Why wasn't the fire put out on the 1st?' Caus...,https://www.latimes.com/california/story/2025-...
307,50,2025-10-07-23-53-21 +0000,nypost,Mother of US Navy medic rescued from Gaza in d...,https://nypost.com/2025/10/07/world-news/mothe...
139,49,2025-10-08-14-44-00 +0000,wsj,"In Japan, as in the U.S., a new leader wants t...",https://www.wsj.com/economy/central-banking/go...
65,43,2025-10-08-18-33-00 +0000,wsj,President Trump called for the jailing of Chic...,https://www.wsj.com/politics/policy/trump-call...
298,36,2025-10-08-00-44-44 +0000,nyt,Athens Democracy Forum: As Democracy Erodes Gl...,https://www.nytimes.com/2025/10/07/world/europ...
240,36,2025-10-08-09-00-00 +0000,wsj,The nation’s top four wind power states all le...,https://www.wsj.com/politics/policy/trumps-win...
115,34,2025-10-08-16-28-37 +0000,nypost,Dad accused of killing girlfriend had sex with...,https://nypost.com/2025/10/08/world-news/dad-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
